<font color="#1DB954">

# Analyzing our Spotify Data

<font color="#1DB954">

## Initialization

<font color="#1DB954">

### **Connecting to Drive**

I use this code flow for all my normal projects. Firstly it primts a message for what enviornment is being used than it stores the directories as variables for further use. In colab since I use that for other subjects as well I have partitioned it accordingly but for my local drive I like to keep it simple. I know I have already created base, data, and archive folders but for output I will run the code below to create it if it does not exist

In [1]:
# Use for normal projects
# In colab since I use that for other subjects as well I have partitioned it accordingly 
# But for my local drive I like to keep it simple
import os
import sys
from pathlib import Path
if 'google.colab' in str(get_ipython()):
  print('Running on Colab')
  from google.colab import drive
  drive.mount('/content/drive') 
  %pip install swifter -qq
  %pip install spotipy --upgrade -qq
  base_folder = Path('/content/drive/MyDrive/colab_notebooks/')
  subject = 'aml'
  data = base_folder/subject/'data/'
  archive = base_folder/subject/'archive/'
  output = base_folder/subject/'output'
  if not Path(base_folder/subject/'output').exists():
    os.makedirs(output,exist_ok=True)
  print(f'Base Folder is {base_folder}')
  print(f'Data Folder is {data}')
  print(f'Archive Folder is {archive}')
  print(f'Output Folder is {output}')
else:
  print('Not Running on Colab')
  # Add the path to the custom-functions directory in the local file system to sys.path
  sys.path.append('C:/Users/Abdul Rauf Maroof/OneDrive/Documents/MSBA/notebooks/spotify_project/custom_functions')
  base_folder = Path('C:/Users/Abdul Rauf Maroof/OneDrive/Documents/MSBA')
  data = base_folder/'data_sets/'
  archive = base_folder/'archive/'
  output = base_folder/'notebooks/spotify_project'/'output'
  if not (base_folder/'notebooks/spotify_project'/'output').exists():
    os.makedirs(output,exist_ok=True)
  print(f'Base Folder is {base_folder}')
  print(f'Data Folder is {data}')
  print(f'Archive Folder is {archive}')
  print(f'Output Folder is {output}')
  print(f'The path to the custom functions is {sys.path[-1]}')
  print(f'The working directory is {os.getcwd()}')


Not Running on Colab
Base Folder is C:\Users\Abdul Rauf Maroof\OneDrive\Documents\MSBA
Data Folder is C:\Users\Abdul Rauf Maroof\OneDrive\Documents\MSBA\data_sets
Archive Folder is C:\Users\Abdul Rauf Maroof\OneDrive\Documents\MSBA\archive
Output Folder is C:\Users\Abdul Rauf Maroof\OneDrive\Documents\MSBA\notebooks\spotify_project\output
The path to the custom functions is C:/Users/Abdul Rauf Maroof/OneDrive/Documents/MSBA/notebooks/spotify_project/custom_functions
The working directory is c:\Users\Abdul Rauf Maroof\OneDrive\Documents\MSBA\notebooks\spotify_project


<font color="#1DB954">

### Import libraries

For the purpose of this project I am using the following libraries:

1. Pandas: Pandas is a popular Python library for data manipulation and analysis. It provides tools for working with tabular data, including data structures for efficient data storage and retrieval, functions for data cleaning and transformation, and methods for summarizing and visualizing data.

2. Numpy: Numpy is a fundamental Python library for scientific computing that provides support for large, multi-dimensional arrays and matrices, as well as a wide range of mathematical functions for working with these arrays. It also provides tools for linear algebra, Fourier analysis, and random number generation.

3. Swifter: Swifter is a library that enables faster processing of Pandas DataFrames by utilizing parallel processing with Dask or Ray. It provides a simple API that allows users to apply Pandas functions to large datasets in parallel.

4. Datetime: Datetime is a Python library that provides classes for working with dates, times, and time intervals. It includes functions for parsing and formatting dates and times, performing arithmetic with dates and times, and handling time zone information.

5. Time: Time is a Python library that provides functions for working with time-related data, including measuring time intervals, sleeping for a specified duration, and retrieving the current time.

6. Zipfile: Zipfile is a Python library for working with ZIP archives, which are collections of one or more files that have been compressed and packaged together. The Zipfile library provides tools for creating, reading, and modifying ZIP archives, as well as extracting files from archives.

7. Json: JSON is a data interchange format that is widely used in web applications and APIs. The json library in Python provides functions for encoding and decoding JSON data, allowing Python objects to be serialized and deserialized into JSON strings.

8. Spotipy: Spotipy is a Python library for working with the Spotify Web API. It provides tools for authenticating with the Spotify API, as well as methods for retrieving information about artists, albums, tracks, playlists, and users.

9. Requests: Requests is a popular Python library for making HTTP requests. It provides a simple and easy-to-use API for sending HTTP requests and handling responses, including support for authentication, cookies, and custom headers.

10. Seaborn: Seaborn is a Python data visualization library that is built on top of Matplotlib. It provides a high-level interface for creating a wide range of statistical graphics, including heatmaps, scatter plots, bar plots, line plots, and more. Seaborn includes a number of built-in themes and color palettes that make it easy to create aesthetically pleasing visualizations. It also provides tools for visualizing statistical relationships in data, such as linear regression, and for creating complex multi-panel figures. Seaborn is commonly used in data science and machine learning applications for exploratory data analysis and presentation of results.

11. Matplotlib: Matplotlib is a Python library that is commonly used for creating high-quality visualizations such as graphs, plots, charts, and histograms. It is an open-source library that allows users to create static, animated, and interactive visualizations in Python.

In [2]:
# Import required libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations
import swifter # speed up apply from pandas
from datetime import datetime, timedelta # to work with datetime and perform operations on it 
import time # to work with time 

# Import zipfile to extract data
# Import JSON to read JSON file
import zipfile
import json

# to connect to spotify api and retrieve data
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
import requests

import seaborn as sns # for data visualization
import matplotlib.pyplot as plt # for data visualization

<font color="#1DB954">

### Define Functions to use

I created some functions to make data extraction and processing easier. These functions helped me to automate a wide array of tasks by making it easier to process and alter datasets as needed

In [3]:
def datetime_from_utc_to_local(utc_datetime):
    """
    Converts a datetime value in UTC timezone to the local timezone.

    Args:
        utc_datetime (datetime.datetime): The datetime value to be converted from UTC to local timezone.

    Returns:
        datetime.datetime: The converted datetime value in local timezone.

    Example:
        >>> datetime_from_utc_to_local(datetime.datetime(2023, 3, 3, 12, 0, 0))
        datetime.datetime(2023, 3, 3, 6, 0)

    Note:
        This function assumes that the system timezone is the local timezone. If the system timezone is not the local timezone,
        you may need to modify the function accordingly to handle the correct timezone offset.
    """
    now_timestamp = time.time()
    offset = datetime.fromtimestamp(now_timestamp) - datetime.utcfromtimestamp(now_timestamp)
    return utc_datetime + offset

In [4]:
def extract_data(text: str, directory: str) -> pd.DataFrame:
    """
    Extracts data from JSON files in a given directory that contain the specified text.

    Args:
        text (str): The text to search for in the filename of the JSON files.
        directory (str): The directory where the JSON files are stored.

    Returns:
        pd.DataFrame: A DataFrame containing the concatenated data from all matching JSON files.

    Example:
        >>> extract_data('2022', '/path/to/data/')
        extracting data from streaming_data_2022.json
        extracting data from playlist_data_2022.json
        <pd.DataFrame>

    Note:
        This function assumes that the JSON files have a '.json' file extension and that the DataFrame can be concatenated
        using the `pd.concat()` function. You may need to modify the function to handle different file extensions or
        different ways of merging the data.
    """
    dataframe = pd.DataFrame()
    for entry in Path(directory).iterdir():
        if 'json' in entry.name and text in entry.name:
            print(f'extracting data from {entry.name}')
            dataframe = pd.concat([dataframe, pd.read_json(entry)], axis=0, ignore_index=True)
    return dataframe

In [5]:
# Gets dataframe last 50 before a time period
def recently_played(sp, limit=50, after=None):
    """
    Returns a pandas DataFrame containing information about the user's recently played tracks.

    Parameters:
    sp (spotipy.Spotify): an authenticated Spotipy client object
    limit (int, optional): the maximum number of results to return (default 50)
    after (datetime, optional): a datetime object specifying the earliest time to include in the results

    Returns:
    pd.DataFrame: a DataFrame containing the following columns:
        - played_at: a datetime object representing the time the track was played
        - end_time: a datetime object representing the time the track finished playing
        - ms_played: an integer representing the duration of the track in milliseconds
        - track_name: a string representing the name of the track
        - artist_name: a string representing the name of the track's artist
        - album_name: a string representing the name of the track's album
        - track_id: a string representing the ID of the track
    """
    if after:
        # convert the timestamp to the ISO 8601 format expected by the API
        after = after.isoformat(timespec='milliseconds')
    recent_tracks = []
    results = sp.current_user_recently_played(limit=limit, after=after)
    for item in results['items']:
        track = item['track']
        played_at = datetime.strptime(item['played_at'], '%Y-%m-%dT%H:%M:%S.%fZ')
        end_time = played_at + timedelta(milliseconds=track['duration_ms'])
        ms_played = track['duration_ms']
        recent_tracks.append({
            'played_at': played_at,
            'end_time': end_time,
            'ms_played': ms_played,
            'track_name': track['name'],
            'artist_name': track['artists'][0]['name'],
            'album_name': track['album']['name'],
            'track_id': track['id']
        })
    return pd.DataFrame(recent_tracks)

In [6]:
def get_track_and_artist_ids(tracks):
    """
    Returns a pandas DataFrame containing information about the tracks and artists in the given list.

    Parameters:
    tracks (list): a list of tuples containing the track name and artist name

    Returns:
    pd.DataFrame: a DataFrame containing the following columns:
        - id: a string representing the ID of the track
        - track_uri: a string representing the URI of the track
        - track_name: a string representing the name of the track
        - artist_id: a string representing the ID of the track's artist
        - artist_name: a string representing the name of the track's artist
        - artist_uri: a string representing the URI of the track's artist
        - album_id: a string representing the ID of the track's album
        - album_name: a string representing the name of the track's album
        - album_uri: a string representing the URI of the track's album

    Example:
    >>> get_track_and_artist_ids([('The Less I Know the Better', 'Tame Impala')])
    """
    track_ids = []
    for track in tracks:
        name, artist = track
        results = sp.search(q='track:' + name + ' artist:' + artist, type='track', limit=50)
        items = results['tracks']['items']
        if items:
            track_id = items[0]['id']
            track_uri = items[0]['uri']
            artist_id = items[0]['artists'][0]['id']
            artist_uri = items[0]['artists'][0]['uri']
            album_id = items[0]['album']['id']
            album_name = items[0]['album']['name']
            album_uri = items[0]['album']['uri']
            track_ids.append((track_id, track_uri ,name, artist_id, artist, artist_uri, album_id, album_name, album_uri))
        else:
            track_ids.append((None, name, artist))
    return pd.DataFrame(track_ids, columns=['id', 'track_uri','track_name', 'artist_id', 'artist_name', 'artist_uri', 'album_id', 'album_name', 'album_uri'])

In [7]:
def time_of_day(datetime_column: str, df):
   """
   Categorizes the time of day for each datetime value in the specified column of a given DataFrame.

   Args:
      datetime_column (str): The name of the column containing datetime values in the DataFrame.
      df (pandas.DataFrame): The DataFrame to extract datetime values from. Default is streaming_data.

   Returns:
      pandas.Categorical: A categorical object containing the time of day for each datetime value in the specified column.
      The categories are 'morning', 'afternoon', 'evening' and 'night', ordered chronologically.

   Example:
      >>> time_of_day('datetime', df=my_data)
      [morning, afternoon, night, ...]
      Categories (4, object): [morning < afternoon < evening < night]
   """
   time_of_day = []
   times = df[datetime_column].dt.hour
   for time in times:
         if time < 6:
            time_of_day.append('night')
         elif time < 12:
            time_of_day.append('morning')
         elif time < 18:
            time_of_day.append('afternoon')
         else:
            time_of_day.append('evening')
   return pd.Categorical(time_of_day, categories=['morning', 'afternoon', 'evening', 'night'], ordered=True)
   

In [8]:
def day_of_week(datetime_column: str, df):
  """
  Extracts the day of the week from each datetime value in the specified column of a given DataFrame.

  Args:
    datetime_column (str): The name of the column containing datetime values in the DataFrame.
    df (pandas.DataFrame): The DataFrame to extract datetime values from. Default is streaming_data.

  Returns:
    pandas.Categorical: A categorical object containing the day of the week for each datetime value in the specified column.
    The categories are 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', and 'Sunday', ordered chronologically.

  Example:
    >>> day_of_week('datetime', df=my_data)
    [Monday, Tuesday, Wednesday, ...]
    Categories (7, object): [Monday < Tuesday < Wednesday < Thursday < Friday < Saturday < Sunday]
  """
  day_of_week = [i.strftime('%A') for i in df[datetime_column]]
  day_of_week = pd.Categorical(day_of_week, categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],ordered=True)
  return day_of_week

In [9]:
def month(datetime_column: str, df):
  """
  Extracts the day of the week from each datetime value in the specified column of a given DataFrame.

  Args:
    datetime_column (str): The name of the column containing datetime values in the DataFrame.
    df (pandas.DataFrame): The DataFrame to extract datetime values from. Default is streaming_data.

  Returns:
    pandas.Categorical: A categorical object containing the day of the week for each datetime value in the specified column.
    The categories are 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', and 'Sunday', ordered chronologically.

  Example:
    >>> day_of_week('datetime', df=my_data)
    [Monday, Tuesday, Wednesday, ...]
    Categories (7, object): [Monday < Tuesday < Wednesday < Thursday < Friday < Saturday < Sunday]
  """
  month = [i.strftime('%B') for i in df[datetime_column]]
  month = pd.Categorical(month, categories=['January','February','March','April','May','June','July','August','September','October','November','December'],ordered=True)
  return month

In [10]:
def get_track_audio_features(tracks):
    """
    Returns a pandas DataFrame containing information about the tracks, artists, and audio features in the given list.

    Parameters:
    tracks (list): a list of tuples containing the track name and artist name

    Returns:
    pd.DataFrame: a DataFrame containing the following columns:
        - id: a string representing the ID of the track
        - track_uri: a string representing the URI of the track
        - track_name: a string representing the name of the track
        - acousticness: a float representing the acousticness of the track
        - danceability: a float representing the danceability of the track
        - energy: a float representing the energy of the track
        - instrumentalness: a float representing the instrumentalness of the track
        - liveness: a float representing the liveness of the track
        - loudness: a float representing the loudness of the track
        - speechiness: a float representing the speechiness of the track
        - tempo: a float representing the tempo of the track
        - valence: a float representing the valence of the track
        - sections: a list of dictionaries containing information about the track's sections
        - segments: a list of dictionaries containing information about the track's segments

    Example:
    >>> get_track_info([('The Less I Know the Better', 'Tame Impala')])
    """
    track_info = []
    for track in tracks:
        name, artist = track
        results = sp.search(q='track:' + name + ' artist:' + artist, type='track')
        items = results['tracks']['items']
        if items:
            track_id = items[0]['id']
            track_uri = items[0]['uri']
            artist_id = items[0]['artists'][0]['id']
            artist_uri = items[0]['artists'][0]['uri']
            features = sp.audio_features(track_uri)[0]
            track_info.append((track_id, track_uri, name, artist_id, artist, artist_uri,
                               features['acousticness'], features['danceability'], features['energy'],
                               features['instrumentalness'], features['liveness'], features['loudness'],
                               features['speechiness'], features['tempo'], features['valence']))
        else:
            track_info.append((None, name, artist))
    return pd.DataFrame(track_info, columns=['id', 'track_uri', 'track_name', 'artist_id','artist_name','artist_uri','acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo','valence'])

In [11]:
def get_track_genres(tracks):
    """
    Returns a pandas DataFrame containing information about the tracks, artists, and audio features in the given list.

    Parameters:
    tracks (list): a list of tuples containing the track name and artist name

    Returns:
    pd.DataFrame: a DataFrame containing the following columns:
        - id: a string representing the ID of the track
        - track_uri: a string representing the URI of the track
        - track_name: a string representing the name of the track
        - artist_id: a string representing the ID of the track's artist
        - artist_name: a string representing the name of the track's artist
        - artist_uri: a string representing the URI of the track's artist
        - album_id: a string representing the ID of the track's album
        - album_name: a string representing the name of the track's album
        - album_uri: a string representing the URI of the track's album
        - artist_genres: a list of strings representing the genres of the track's artist
        - album_genres: a list of strings representing the genres of the track's album

    Example:
    >>> get_track_info([('The Less I Know the Better', 'Tame Impala')])
    """
    track_info = []
    for track in tracks:
        name, artist = track
        results = sp.search(q='track:' + name + ' artist:' + artist, type='track')
        items = results['tracks']['items']
        if len(items) > 0:
            track_id = items[0]['id']
            track_uri = items[0]['uri']
            artist_id = items[0]['artists'][0]['id']
            artist_uri = items[0]['artists'][0]['uri']
            artist_genres = sp.artist(artist_id)['genres']
            track_info.append((track_id, track_uri, name, artist_id, artist, artist_uri, artist_genres))
        else:
            track_info.append((None, name, artist))
    return pd.DataFrame(track_info, columns=['id', 'track_uri', 'track_name', 'artist_id', 'artist_name', 'artist_uri','artist_genres'])


<font color="#1DB954">

### Setting up API for data extraction

I got most of my data from spotify however there were certain features that I wanted to add that were not available in the dataset such as genres, audio features etc. I also want to extract song ids and spotify's own unique resource identifier code

This cell is to update my json credentials file in case of changes

In [12]:
# with open(data/'credentials.json') as f:
#     credentials = json.load(f)

# print(credentials)

# with open(data/'credentials.json', 'w') as f:
#     json.dump(credentials, f)

The code below allows us to access the api and draw data from it as needed

In [602]:
# Get credentials from Spotify for Developers
with open(data/'credentials.json') as f:
    cred = json.load(f)

# define the read scope
scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="3b6c24d8b63a40f891504976c1872909",
                                               client_secret="44da19a313634041a2f3e2e225680ca7",
                                               redirect_uri=cred['redirect_uri'],
                                               scope=scope))

<font color="#1DB954">

## Data preparation

<font color="#1DB954">

### Unzip data


In [13]:
zips = [x for x in archive.iterdir() if 'zip' in x.name]
for entry in archive.iterdir():
  if 'zip' in entry.name:
    print(entry.name)

air_quality_data.zip
my_spotify_data.zip
my_spotify_data_2.zip
Spotify-20230303T164824Z-001.zip
stock_data.zip


In [14]:
file = zips[1]

with zipfile.ZipFile(file,'r') as f:
  print(f.namelist())
  if not Path(data/'MyData').exists():
    f.extractall(data)
    print('\n',f'Data was extracted to {data}')
  else:
    print('\n',f'Data already extracted to {data}')

['MyData/DuoNewFamily.json', 'MyData/FamilyPlan.json', 'MyData/Follow.json', 'MyData/Identifiers.json', 'MyData/Identity.json', 'MyData/Inferences.json', 'MyData/Marquee.json', 'MyData/Payments.json', 'MyData/Playlist1.json', 'MyData/SearchQueries.json', 'MyData/StreamingHistory0.json', 'MyData/StreamingHistory1.json', 'MyData/StreamingHistory2.json', 'MyData/StreamingHistory3.json', 'MyData/Userdata.json', 'MyData/YourLibrary.json', 'MyData/', 'MyData/Read_Me_First.pdf']

 Data already extracted to C:\Users\Abdul Rauf Maroof\OneDrive\Documents\MSBA\data_sets


<font color="#1DB954">

### Create Pandas DataFrame from JSON

<font color="#1DB954">

Now I'll extract data from the json files as needed and process it for further analysis by adding columns to perform joins and creating secondary tables

In [605]:
# changing the direcotry to match the data
spotify_data = data/'MyData'
for entry in spotify_data.iterdir():
  if 'json' in entry.name:
    print(entry.name)

DuoNewFamily.json
FamilyPlan.json
Follow.json
Identifiers.json
Identity.json
Inferences.json
Marquee.json
Payments.json
Playlist1.json
SearchQueries.json
StreamingHistory0.json
StreamingHistory1.json
StreamingHistory2.json
StreamingHistory3.json
Userdata.json
YourLibrary.json


<font color="#1DB954">

The files that important for the purpose of our analysis are as follows:

1. Streaming Data
2. Playlists
3. Library

<font color="#1DB954">

##### Streaming Data

In [606]:
streaming_data = extract_data('Streaming',spotify_data)
streaming_data.head()

extracting data from StreamingHistory0.json
extracting data from StreamingHistory1.json
extracting data from StreamingHistory2.json
extracting data from StreamingHistory3.json


,endTime,artistName,trackName,msPlayed
0,2022-02-27 10:01,Billie Eilish,bellyache,1680
1,2022-02-27 10:01,Sam Smith,Like I Can,2130
2,2022-02-27 10:01,Ruth B.,Dandelions,4240
3,2022-02-27 10:03,The Weeknd,The Hills,125510
4,2022-02-27 10:04,The Weeknd,Call Out My Name,11760


<font color="#1DB954">

##### Playlist data

In [607]:
# Extracting data from Playlist1.json
playlist_df = extract_data('Playlist', spotify_data); display(playlist_df.head(4))
print('\n')
# from the look of it seems that I further need to work on the playlist file to properly extract data
playlist_dict = pd.DataFrame([row for row in playlist_df.playlists]); display(playlist_dict.head(4))
print('\n')
# Create a breaking point because I want to keep a tabel of all my playlists
# I will also drop the items column because it contains a list of all the songs in the playlist 
# and I will extract that data in a different way
playlist_core_df = playlist_dict
playlist_core_df.drop(['items'], axis=1, inplace=True)
#change the lastModifiedDate column to datetime format
playlist_core_df['lastModifiedDate'] = pd.to_datetime(playlist_core_df['lastModifiedDate'])

extracting data from Playlist1.json


,playlists
0,"{'name': 'Hozier’s Lullaby ', 'lastModifiedDat..."
1,"{'name': 'Mile High View', 'lastModifiedDate':..."
2,"{'name': 'Recent Favorites', 'lastModifiedDate..."
3,"{'name': 'Cyberpunk', 'lastModifiedDate': '202..."


,name,lastModifiedDate,items,description,numberOfFollowers
0,Hozier’s Lullaby,2023-02-23,"[{'track': {'trackName': 'NFWMB', 'artistName'...",None,0
1,Mile High View,2023-02-03,"[{'track': {'trackName': 'goosebumps', 'artist...",None,0
2,Recent Favorites,2022-12-11,[{'track': {'trackName': 'Blood Upon the Snow'...,None,0
3,Cyberpunk,2022-07-06,"[{'track': {'trackName': 'Teenagers', 'artistN...",None,0


In [608]:
# firstly I have all the playlists I have now I will repeat the process to create a dataframe for each playlist 
# and combine that into one with each playlist
playlist_dict = pd.DataFrame([row for row in playlist_df.playlists])
for i, value in enumerate(playlist_dict['items']):
  globals()[f'playlist_{i+1}'] = value

In [609]:
# This loop will build on the previous loop and create a dataframe for each playlist
# which I will concatenate into one dataframe
for i in range(1, 21):
    df_name = f'playlist_{i}'
    df = globals()[df_name]
    df_1 = pd.DataFrame(df)
    df_2 = pd.DataFrame([row for row in df_1['track']])
    df_2['playlist_name'] = playlist_dict.iloc[i-1]['name']
    globals()[f'playlist_df_{i}'] = df_2.join(df_1).drop('track', axis=1)

playlist_contents_df = pd.concat([globals()[f'playlist_df_{i}'] for i in range(1, 21)], ignore_index=True)
# change the addedDate column to datetime format
playlist_contents_df['addedDate'] = pd.to_datetime(playlist_contents_df['addedDate'])

#check the data types and drop the columns that are not needed
playlist_contents_df.info()
playlist_contents_df.drop(['episode', 'localTrack'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1227 entries, 0 to 1226
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   trackName      1227 non-null   object        
 1   artistName     1227 non-null   object        
 2   albumName      1227 non-null   object        
 3   trackUri       1227 non-null   object        
 4   playlist_name  1227 non-null   object        
 5   episode        0 non-null      object        
 6   localTrack     0 non-null      object        
 7   addedDate      1227 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 76.8+ KB


In [610]:
# change the addedDate column to datetime format
playlist_contents_df['addedDate'] = pd.to_datetime(playlist_contents_df['addedDate'])

# create a count dataframe to join with the playlists core table to get a count of the number of songs in each playlist
count = pd.DataFrame(playlist_contents_df.groupby('playlist_name')['playlist_name'].value_counts()).reset_index(level=1, drop=True).rename(columns={'name':'count'})

#finally join the count dataframe with the playlist core dataframe
playlist_core_df = playlist_core_df.join(count, on='name', how='left')

In [611]:
# Keep in mind it is better to run certain cells seperately to avoid errors
playlist_core_df.rename(columns={'playlist_name':'count'}, inplace=True)
playlist_core_df.rename(columns={'name':'playlist_name'}, inplace=True)

In [612]:
# I'll drop number of followers and description because they are not needed and are empty or insignificant
playlist_core_df.drop(['description', 'numberOfFollowers'], axis=1, inplace=True)
playlist_core_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   playlist_name     20 non-null     object        
 1   lastModifiedDate  20 non-null     datetime64[ns]
 2   count             20 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 608.0+ bytes


<font color="#1DB954">

#### Library Data

In [613]:
# After some analysis I settled on these keys as important while the others were empty
file_json = spotify_data / 'YourLibrary.json'
required_keys = ['tracks', 'shows', 'artists']
# Opening JSON file
with open(file_json) as f:
  
    # returns JSON object as 
    # a dictionary
    libraries = json.load(f)

    # Iterating through the json keys
    for key in libraries.keys():
        if key in required_keys:
            globals()[f"library_{key}"] = pd.DataFrame(libraries[key])

In [614]:
display(library_tracks.head())
display(library_artists.head())
display(library_shows.head())

,artist,album,track,uri
0,Natalia Lacunza,Otras Alas,Olivia,spotify:track:0okyvelhnnloZWHv6DwgSM
1,Nusrat Fateh Ali Khan,Shahen-Shah,Kali Kali Zulfon Ke Phande Nah Dalo,spotify:track:41p2AWpXt0QF4GFlsxVowq
2,Marshmello,BIBA,BIBA,spotify:track:0GVBPuCT3pBrXCdY3Uiy5B
3,Brothers + Company,Run,Take This Love,spotify:track:74ehQPUbiEVkUM0UdmkVzc
4,Pomme,les failles,grandiose,spotify:track:3NakEPSaVKsqqimJW66xCc


,name,uri
0,Ali Sethi,spotify:artist:3NegWDGp038A3FIi3gSYzl
1,Arctic Monkeys,spotify:artist:7Ln80lUS6He07XvHI8qqHH
2,Camila Cabello,spotify:artist:4nDoRrQiYLoBzwC5BhVJzF
3,Childish Gambino,spotify:artist:73sIBHcqh3Z3NyqHKZ7FOL
4,Coldplay,spotify:artist:4gzpq5DPGxSnKTe4SA8HAU


,name,publisher,uri
0,A History of the World in 100 Objects,BBC Radio 4,spotify:show:01J8pPwYAt3NEvAkkuDOSs
1,Archetypes,Archewell Audio,spotify:show:6UfyXZgVAUX1UzF8j5L72t
2,Batman Unburied,Warner Bros. / Spotify Studios,spotify:show:3pUWoZ6fC2qA02D3X0CeMb
3,Batman: The Audio Adventures,HBO Max,spotify:show:6YOANXuJ8AAtB2bFilDm6g
4,Chasing Life,CNN,spotify:show:3nKvLT48YbbRQoCYQ4ern1


<font color="#1DB954">

### Retriving Data from the API

<font color="#1DB954">

As previously mentioned there is data I need from the API and I will be using some custom functions to extract that. It is important to note that after the data has been extracted I have commented out the because I don not need to run these again and again rather the data is appropriately stored in the ```output folder ```

In [615]:
# This code block checks to see if the tracks.csv file exists and if it does it loads it
# if it does not exist it prints a message to the user to run the search
# I am creating the search list here that is used in to extract data function

search  = list(set(zip(streaming_data['trackName'], streaming_data['artistName'])))
if Path(output/'tracks.csv').exists():
    tracks = pd.read_csv(output/'tracks.csv') 
    print(f"Data from tracks has been loaded with total records:\n{len(tracks)}\n")
if Path(output/'audio_features.csv').exists():
    audio_features = pd.read_csv(output/'audio_features.csv')
    print(f"Data from audio_features has been loaded with total records:\n{len(audio_features)}\n")
if Path(output/'genres.csv').exists():
    genres = pd.read_csv(output/'genres.csv')
    print(f"Data from genres has been loaded with total records:\n{len(genres)}\n")
else:
    print(f'Looks like you have not run the search yet, please run the search and try again\n')

Data from tracks has been loaded with total records:
6152

Data from audio_features has been loaded with total records:
6152

Data from genres has been loaded with total records:
6152



<font color="#1DB954">

The code block below is to extract data from the API for the corresponding tracks so that I can collect more data such as audio features and genres correctly. I only need to run this once to collect the data I need hence I have now commented out this block 

----

In [616]:
# for i in range(0, len(search), 50):
#     print(f'Function running {i/50} times')
#     tracks = pd.DataFrame()
#     track_ids = get_track_and_artist_ids(search[i:i+50])
#     tracks = tracks.append(track_ids)
#     tracks.to_csv(output/'tracks.csv', mode='a', header=False)

# tracks = pd.read_csv(output/'tracks.csv',  names=['id', 'track_uri','track_name', 'artist_id', 'artist_name', 'artist_uri', 'album_id', 'album_name', 'album_uri'])
# tracks['key'] = tracks['track_name'] + ':'+ tracks['artist_name']
# tracks.reset_index(drop=True,inplace=True)
# tracks.to_csv(output/'tracks.csv', index=False)

In [617]:
# for i in range(0, len(search), 50):
#     print(f'Function running {i/50} times')
#     audio_features = pd.DataFrame()
#     audio_feature_data = get_track_audio_features(search[i:i+50])
#     audio_features = audio_features.append(audio_feature_data)
#     audio_features.to_csv(output/'audio_features.csv', mode='a', header=False)

# audio_features = pd.read_csv(output/'audio_features.csv',  names=['id', 'track_uri', 'track_name', 'artist_id','artist_name','artist_uri','acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo','valence'])
# audio_features['key'] = audio_features['track_name'] + ':'+ audio_features['artist_name']
# audio_features.reset_index(drop=True,inplace=True)
# audio_features.to_csv(output/'audio_features.csv', index=False)

In [618]:
# for i in range(0, len(search), 50):
#     print(f'Function running {i/50} times')
#     genres = pd.DataFrame()
#     genres_data = get_track_genres(search[i:i+50])
#     genres = genres.append(genres_data)
#     genres.to_csv(output/'genres.csv', mode='a', header=False)

# genres = pd.read_csv(output/'genres.csv',  names=['id', 'track_uri', 'track_name', 'artist_id', 'artist_name', 'artist_uri','genres_albums'])
# genres['key'] = genres['track_name'] + ':'+ genres['artist_name']
# genres.reset_index(drop=True,inplace=True)
# genres.to_csv(output/'genres.csv', index=False)

In [619]:
# I will finally store the unedited data in a folder called my_spotify_api_data
# This is to ensure that I can always go back to the original data

if (data/'my_spotify_api_data').exists():
    tracks.to_csv(data/'my_spotify_api_data/tracks.csv', index=False)
    audio_features.to_csv(data/'my_spotify_api_data/audio_features.csv', index=False)
    genres.to_csv(data/'my_spotify_api_data/genres.csv', index=False)
else:
    os.makedirs(data/'my_spotify_api_data', exist_ok=True)
    tracks.to_csv(data/'my_spotify_api_data/tracks.csv', index=False)
    audio_features.to_csv(data/'my_spotify_api_data/audio_features.csv', index=False)
    genres.to_csv(data/'my_spotify_api_data/genres.csv', index=False)

----

<font color="#1DB954">

### Cleaning and processing API data

<font color="#1DB954">

Once I have extracted data from the api I can now begin to process and clean it for further analysis

In [620]:
# Check to make sure the data is loaded correctly
display(tracks.info())
print('\n')
display(audio_features.info())
print('\n')
display(genres.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6152 entries, 0 to 6151
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           5586 non-null   object
 1   track_uri    6152 non-null   object
 2   track_name   6152 non-null   object
 3   artist_id    5586 non-null   object
 4   artist_name  5586 non-null   object
 5   artist_uri   5586 non-null   object
 6   album_id     5586 non-null   object
 7   album_name   5586 non-null   object
 8   album_uri    5586 non-null   object
 9   key          5586 non-null   object
dtypes: object(10)
memory usage: 480.8+ KB


None



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6152 entries, 0 to 6151
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                5586 non-null   object 
 1   track_uri         6152 non-null   object 
 2   track_name        6152 non-null   object 
 3   artist_id         5586 non-null   object 
 4   artist_name       5586 non-null   object 
 5   artist_uri        5586 non-null   object 
 6   acousticness      5586 non-null   float64
 7   danceability      5586 non-null   float64
 8   energy            5586 non-null   float64
 9   instrumentalness  5586 non-null   float64
 10  liveness          5586 non-null   float64
 11  loudness          5586 non-null   float64
 12  speechiness       5586 non-null   float64
 13  tempo             5586 non-null   float64
 14  valence           5586 non-null   float64
 15  key               5586 non-null   object 
dtypes: float64(9), object(7)
memory usage: 7

None



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6152 entries, 0 to 6151
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             5586 non-null   object
 1   track_uri      6152 non-null   object
 2   track_name     6152 non-null   object
 3   artist_id      5586 non-null   object
 4   artist_name    5586 non-null   object
 5   artist_uri     5586 non-null   object
 6   genres_albums  5586 non-null   object
 7   key            5586 non-null   object
dtypes: object(8)
memory usage: 384.6+ KB


None

In [621]:
# I now explore the tracks data to see how many tracks were correctly identified
# I can see 
tracks['track_name'].count() == len(search)
print(f"I can see that {round((1 - (tracks['id'].count() / tracks['track_name'].count())) * 100,2)}% of the tracks were not identified\n")

I can see that 9.2% of the tracks were not identified



In [622]:
# I have extracted the data for tracks that had no Id. Based on the name I can see that these are actually podcasts
podcasts = tracks[tracks['id'].isna()].reset_index(drop=True)
podcasts  = podcasts[['track_uri','track_name']].rename({'track_uri':'episode', 'track_name':'podcast'}, axis=1)

In [623]:
# Interestingly there are some songs mixed into the podcasts but a majority of them are podcasts
# So I will drop the songs and keep the podcasts
podcasts.groupby('podcast').count().sort_values(by='episode', ascending=False).head(20)
podcasts = podcasts[podcasts.groupby('podcast')['podcast'].transform('count') > 1]
# check if podcast is actually an artist by cross referencing the artist table
podcasts = podcasts[~podcasts['podcast'].isin(tracks['artist_name'])]
podcasts.groupby('podcast').count().sort_values(by='episode', ascending=False).head(20)
podcasts['key'] = podcasts['podcast'] + podcasts['episode']

In [624]:
# Now I will drop the podcasts from the tables because we can see that features for these are not available
tracks = tracks[tracks['id'].notna()].reset_index(drop=True)
audio_features = audio_features[audio_features['id'].notna()].reset_index(drop=True)
genres = genres[genres['id'].notna()].reset_index(drop=True)

In [625]:
# We finally check to make sure that the tables are consistent with each other
display(tracks.info())
print('\n')
display(audio_features.info())
print('\n')
display(genres.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5586 entries, 0 to 5585
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           5586 non-null   object
 1   track_uri    5586 non-null   object
 2   track_name   5586 non-null   object
 3   artist_id    5586 non-null   object
 4   artist_name  5586 non-null   object
 5   artist_uri   5586 non-null   object
 6   album_id     5586 non-null   object
 7   album_name   5586 non-null   object
 8   album_uri    5586 non-null   object
 9   key          5586 non-null   object
dtypes: object(10)
memory usage: 436.5+ KB


None



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5586 entries, 0 to 5585
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                5586 non-null   object 
 1   track_uri         5586 non-null   object 
 2   track_name        5586 non-null   object 
 3   artist_id         5586 non-null   object 
 4   artist_name       5586 non-null   object 
 5   artist_uri        5586 non-null   object 
 6   acousticness      5586 non-null   float64
 7   danceability      5586 non-null   float64
 8   energy            5586 non-null   float64
 9   instrumentalness  5586 non-null   float64
 10  liveness          5586 non-null   float64
 11  loudness          5586 non-null   float64
 12  speechiness       5586 non-null   float64
 13  tempo             5586 non-null   float64
 14  valence           5586 non-null   float64
 15  key               5586 non-null   object 
dtypes: float64(9), object(7)
memory usage: 6

None



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5586 entries, 0 to 5585
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             5586 non-null   object
 1   track_uri      5586 non-null   object
 2   track_name     5586 non-null   object
 3   artist_id      5586 non-null   object
 4   artist_name    5586 non-null   object
 5   artist_uri     5586 non-null   object
 6   genres_albums  5586 non-null   object
 7   key            5586 non-null   object
dtypes: object(8)
memory usage: 349.2+ KB


None

<font color="#1DB954">

## EDA and Data Cleaning

In [626]:
streaming_data['endTime'] = pd.to_datetime(streaming_data['endTime'])
streaming_data['endTime'] = datetime_from_utc_to_local(streaming_data['endTime'])
streaming_data.sort_values(by='endTime')

,endTime,artistName,trackName,msPlayed
0,2022-02-27 04:01:00,Billie Eilish,bellyache,1680
1,2022-02-27 04:01:00,Sam Smith,Like I Can,2130
2,2022-02-27 04:01:00,Ruth B.,Dandelions,4240
3,2022-02-27 04:03:00,The Weeknd,The Hills,125510
4,2022-02-27 04:04:00,The Weeknd,Call Out My Name,11760
...,...,...,...,...
36115,2023-02-27 15:21:00,Post Malone,Circles,22801
36116,2023-02-27 15:21:00,Paul Anka,Put Your Head on My Shoulder,15418
36118,2023-02-27 15:21:00,Regard,Ride It,5387
36119,2023-02-27 15:22:00,Lana Del Rey,Fuck it I love you,3784


In [627]:
streaming_data['key'] = streaming_data['artistName']+':'+streaming_data['trackName']
streaming_data['minutes_played'] = streaming_data['msPlayed'].swifter.apply(lambda x: (x / 60000)) # there are 60,000 ms in a minute
streaming_data['seconds_played'] = streaming_data['msPlayed'].swifter.apply(lambda x: (x / 1000)) # there are 1000 ms in a second
streaming_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36121 entries, 0 to 36120
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   endTime         36121 non-null  datetime64[ns]
 1   artistName      36121 non-null  object        
 2   trackName       36121 non-null  object        
 3   msPlayed        36121 non-null  int64         
 4   key             36121 non-null  object        
 5   minutes_played  36121 non-null  float64       
 6   seconds_played  36121 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 1.9+ MB


In [630]:
streaming_data['time_of_day'] = time_of_day('endTime',df=streaming_data)
streaming_data['day_of_week'] = day_of_week('endTime',df=streaming_data)
streaming_data['month'] = month('endTime',df=streaming_data)

In [629]:
pd.pivot_table(streaming_data, index='time_of_day', values='trackName', aggfunc='count',margins=True)

,trackName
time_of_day,
morning,6993
afternoon,12873
evening,9809
night,6446
All,36121


<font color="#1DB954">

## Save all files to CSV

In [535]:
# Save streaming data
streaming_data.to_csv(output/'streaming_data.csv',index=False)
# Save playlist core data
playlist_core_df.to_csv(output/'playlist_core_data.csv',index=False)
# Save playlist contents
playlist_contents_df.to_csv(output/'playlist_contents_data.csv',index=False)
# Save library_artists
library_artists.to_csv(output/'library_artists.csv',index=False)
# Save library_shows
library_shows.to_csv(output/'library_shows.csv',index=False)
# Save library_tracks
library_tracks.to_csv(output/'library_tracks.csv',index=False)
# Save podcasts
podcasts.to_csv(output/'podcasts.csv', index=False)